Ноутбук составлен по мотивам https://github.com/Yorko/mlcourse.ai/blob/master/jupyter_english/topic04_linear_models/topic4_linear_models_part4_good_bad_logit_movie_reviews_XOR.ipynb

# Логистическая регрессия. Пример.

Поработаем с данными IMDB movie reviews.

**Скачайте данные по ссылке https://www.dropbox.com/s/d9fadkx9vi3kw9o/aclImdb_v1.tar.gz?dl=0**

В этом датасете 25000 отзывов на фильмы: 12500 положительных и 12500 отрицательных. Мы хотим решить задачу классификации отзывов.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

Загрузим датасет.

In [ ]:
from sklearn.datasets import load_files

PATH_TO_IMDB = './aclImdb'

reviews_train = load_files(os.path.join(PATH_TO_IMDB, "train"),
                           categories=['pos', 'neg'])
text_train, y_train = reviews_train.data, reviews_train.target

reviews_test = load_files(os.path.join(PATH_TO_IMDB, "test"),
                          categories=['pos', 'neg'])
text_test, y_test = reviews_test.data, reviews_test.target

In [ ]:
print("Number of documents in training data: %d" % len(text_train))
print(np.bincount(y_train))
print("Number of documents in test data: %d" % len(text_test))
print(np.bincount(y_test))

Примеры отзывов

In [ ]:
text_train[1]

In [ ]:
y_train[1]

In [ ]:
text_train[2]

In [ ]:
y_train[2]

### Для генерации признаков используем CountVectorizer

![How it works](CV.png)

![How it works](CV2.png)

In [ ]:
cv = CountVectorizer()
cv.fit(text_train)

len(cv.vocabulary_)

Посмотрим на слова в получившимся словаре.

In [ ]:
print(cv.get_feature_names()[:50])
print(cv.get_feature_names()[50000:50050])

Закодируем предложения индексами слов из словаря.

In [ ]:
X_train = cv.transform(text_train)
X_train

Посмотрим, что получилось

In [ ]:
print(text_train[19723])

In [ ]:
X_train[19723].nonzero()[1]

In [ ]:
X_test = cv.transform(text_test)

## Обучим логистическую регрессию

In [ ]:
logit = LogisticRegression(solver='lbfgs', n_jobs=-1, random_state=7)
logit.fit(X_train, y_train)

Посмотрим на качество

In [ ]:
logit.score(X_train, y_train), logit.score(X_test, y_test)

**Нарисуем наибольшие по модулю веса**

In [ ]:
def visualize_coefficients(classifier, feature_names, n_features=25):

    coef = classifier.coef_.ravel()
    positive_coefficients = np.argsort(coef)[-n_features:]
    negative_coefficients = np.argsort(coef)[:n_features]
    all_coefs = np.hstack([negative_coefficients, positive_coefficients])

    plt.figure(figsize=(15, 5))
    colors = ["red" if c < 0 else "blue" for c in coef[all_coefs]]
    plt.bar(np.arange(2*n_features), coef[all_coefs], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(1, 1+2*n_features), feature_names[all_coefs], rotation=60, ha="right")
    
visualize_coefficients(logit, cv.get_feature_names())

Подберем параметр регуляризации в логистической регрессии.

In [ ]:
from sklearn.pipeline import make_pipeline

pipe_logit = make_pipeline(CountVectorizer(),
                           LogisticRegression(solver='lbfgs', random_state=123))

pipe_logit.fit(text_train, y_train)
pipe_logit.score(text_test, y_test)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV

param_grid = {'logisticregression__C': np.logspace(-5, 0, 6)}
grid_logit = GridSearchCV(pipe_logit, 
                          param_grid, 
                          return_train_score=True, 
                          n_jobs=1,
                          cv=3)

grid_logit.fit(text_train, y_train)

In [ ]:
grid_logit.best_params_, grid_logit.best_score_

Результаты подбора параметра по кросс-валидации

In [ ]:
plt.plot(grid_logit.param_grid['logisticregression__C'], grid_logit.cv_results_['mean_train_score'],
        color='green', label='train')
plt.plot(grid_logit.param_grid['logisticregression__C'], grid_logit.cv_results_['mean_test_score'],
        color='red', label='test')
plt.legend()

In [ ]:
grid_logit.score(text_test, y_test)

### Нарисуем ROC-кривую и вычислим ROC-AUC на тестовых данных.

In [ ]:
from sklearn import metrics

pred = grid_logit.predict_proba(text_test)[::,1]

fpr, tpr, _ = metrics.roc_curve(y_test, pred)

auc = metrics.roc_auc_score(y_test, pred)

plt.plot(fpr,tpr,label="test_data, auc="+str(auc))
plt.legend(loc=4)
plt.show()

#### Задание

Нарисуйте на одном графике ROC-кривые для train и для test. Добавьте на график значения ROC-AUC.

In [ ]:
#your code here

### Нарисуем матрицу ошибок.

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
 
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
 
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
 
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
ans_test = grid_logit.predict(text_test)

plot_confusion_matrix(confusion_matrix(y_test, ans_test), classes=['0','1'],
                        title='Confusion matrix, without normalization')

#### Задание

Добавим в условие задачи, что мы хотим, чтобы как можно меньше положительных отзывов алгоритм предсказывал, как отрицательные. 
Для этого подберите порог вероятности таким образом, чтобы False Negative элементов на тесте было не больше 1000.

Нарисуйте полученную матрицу ошибок.

Выведите значение ROC-AUC.

Выведите значения precision, recall и f1-score (все они находятся в sklearn.metrics).

In [ ]:
#your code here

# Когда линейные модели работают плохо?

![How it works](XOR.png)

Создадим данные для задачи XOR

In [ ]:
rng = np.random.RandomState(0)
X = rng.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired);

Напишем функцию, которая рисует разделяющую границу, проведенную классификатором

In [ ]:
def plot_boundary(clf, X, y, plot_title):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50),
                     np.linspace(-3, 3, 50))
    clf.fit(X, y)
    # plot the decision function for each datapoint on the grid
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1]
    Z = Z.reshape(xx.shape)

    image = plt.imshow(Z, interpolation='nearest',
                           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                           aspect='auto', origin='lower', cmap=plt.cm.PuOr_r)
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2,
                               linetypes='--')
    plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.axis([-3, 3, -3, 3])
    plt.colorbar(image)
    plt.title(plot_title, fontsize=12);

In [ ]:
plot_boundary(LogisticRegression(solver='lbfgs'), X, y,
              "Logistic Regression, XOR problem")

Мы видим, что задача решена плохо. Попробуем добавить в качестве признаков полиномиальные признаки степени 2.

Теперь у нас будут признаки не только $(1, x_1, x_2)$, но и $1, x_1, x_2, x_1^2, x_1x_2, x_2^2$

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

logit_pipe = Pipeline([('poly', PolynomialFeatures(degree=2)), 
                       ('logit', LogisticRegression(solver='lbfgs' ))])

In [ ]:
plot_boundary(logit_pipe, X, y,
              "Logistic Regression + quadratic features. XOR problem")

**Таким образом, при добавлении новых нелинейных признаков с помощью линейного классификатора можно решить линейно неразделимую задачу.**

# Задача классификации: есть ли у человека болезнь сердца.

Вам необходимо предсказать, болен пациент или здоров (target) и добиться наиболее высокого качества предсказания (accuracy). Можно использовать любые методы работы с данными (ограничение: классификатор - логистическая регрессия).

Обучение необходимо провести на данных heart_train.csv, а итоговое качество модели проверяется на heart_test.csv.

В качестве результата домашнего задания вам необходимо прислать код вашей финальной модели и csv-файл с предсказаниями на данных heart_test.csv. Метрика качества - accuracy.


### Описание данных

age - age in years

sex - (1 = male; 0 = female)

cpchest - pain type

trestbpsresting - blood pressure (in mm Hg on admission to the hospital)

cholserum - cholestoral in mg/dl

fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)

restecg - resting electrocardiographic results

thalach - maximum heart rate achieved

exang - exercise induced angina (1 = yes; 0 = no)

oldpeak - ST depression induced by exercise relative to rest

slope - the slope of the peak exercise ST segment

ca - number of major vessels (0-3) colored by flourosopy

thal - 3 = normal; 6 = fixed defect; 7 = reversable defect

target - 1 or 0

In [ ]:
import pandas as pd

df = pd.read_csv("heart_train.csv")
df.head()

Начнём с "бездумного" машинного обучения. Разбейте выборку на тренировочную и валидационную части (20% данных на валидацию), обучите логистическую регрессию с дефолтными параметрами и выведите на экран метрики качества на train и на test.

Постройте ROC-кривую.

Визуализируйте матрицу ошибок.

In [ ]:
#your code here

Теперь серьёзно подойдем к задаче.

### Шаги, которые нужно сделать:

1) Разведочный анализ данных (построение графиков, нахождение корреляций)

2) Различные способы кодирования данных

3) Генерация новых признаков (полиномиальные признаки, другие функции от признаков).

4*) Очистка данных (возможно, в данных есть аномальные значения, которые будут видны при визуализации), с ними надо поработать, можно удалить

5) Подбор гиперпараметров логистической регрессии по кросс-валидации

6) Проверка модели на переобученность

7) Построение ROC-кривой и вычисление ROC-AUC на тренировочной и тестовой выборках. Вычисление accuracy, precision, recall, f1-score.

In [ ]:
#your code here